In [165]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import IPython

In [166]:
def normalize(v):
  norm = np.linalg.norm(v)
  if norm == 0:
    return v
  return v / norm

In [167]:
class Car(ap.Agent):
  def setup(self):
    self.position = (0,0)   # x | y
    self.direction = (0,0)  # x | y
    self.velocity = 0.0     # float
  
  def setup_pos(self, av):
    self.avenue = av
    # self.neighbors = av.neighbors
    self.pos = av.positions[self]
  
  def update_velocity(self):
    ndim = self.p.ndim
    v1 = np.zeros(ndim)
    v2 = np.zeros(ndim)
    v3 = np.zeros(ndim)
    v4 = np.zeros(ndim)
    self.velocity += v1 + v2 + v3 + v4
    self.velocity = normalize(self.velocity)
  
  def update_position(self):
    self.avenue.move_by(self, self.velocity)

In [168]:
class Traffic_Light(ap.Agent):
  def setup(self):
    self.position = (0,0)   # x | y
    self.direction = (0,0)  # x | y
    self.state = 0          # 0: red | 1: yellow | 2: green
    
  def setup_pos(self, av):
    self.avenue = av
    # self.neighbors = av.neighbors
    self.pos = av.positions[self]

In [169]:
class Model(ap.Model):
  def setup(self):
    self.avenue = ap.Space(self, shape=[self.p.size]*self.p.ndim)
    
    self.cars = ap.AgentList(self, self.p.population, Car)
    #self.cars[0].pos = (20, 10)
    #self.cars[1].pos = (40, 20)
    
    self.traffic_lights = ap.AgentList(self, 1, Traffic_Light)
    #self.traffic_lights[0].pos = (10, 40)
    
    self.avenue.add_agents(self.cars)
    self.avenue.add_agents(self.traffic_lights)
    
    self.cars.setup_pos(self.avenue)
    self.traffic_lights.setup_pos(self.avenue)
    
  def step(self):
    self.cars.update_velocity()
    self.cars.update_position()

In [170]:
parameters2D = {
    'size': 50,
    'seed': 123,
    'steps': 200,
    'ndim': 2,
    'population': 2,
}

In [171]:
def animation_plot_single(m, ax):
    ndim = m.p.ndim
    ax.set_title(f"Avenue Simulation {ndim}D t={m.t}")
    pos = m.avenue.positions.values()
    pos = np.array(list(pos)).T  # Transform
    print(pos)
    ax.scatter(*pos, s=100, c='black')
    ax.set_xlim(0, m.p.size)
    ax.set_ylim(0, m.p.size)
    if ndim == 3:
        ax.set_zlim(0, m.p.size)
    ax.set_axis_off()

In [172]:
def animation_plot(m, p):
    projection = '3d' if p['ndim'] == 3 else None
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111, projection=projection)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20))

In [173]:
animation_plot(Model, parameters2D)

[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]
